In [ ]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import os
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# GET BEST PARAMS

os.chdir('/content/drive/My Drive/Digit Recognizer/Parameters')
params = {}
for l in range(1,4):
    params['W' + str(l)] = np.array(pd.read_csv('params W' + str(l) + '.csv').values)
    params['b' + str(l)] = np.array(pd.read_csv('params b' + str(l) + '.csv').values)

In [ ]:
def create_op_vec(i):
    op = np.zeros((10,1))
    op[i] = 1
    return op

In [ ]:
# loading data
os.chdir('/content/drive/My Drive/Digit Recognizer')
train_csv = pd.read_csv('train.csv')
data = np.array(train_csv.values)

In [ ]:
# gathering data
Y = data.T[0].T
X = np.uint8(data.T[1:785].T)

# generating test and train sets
x_train = X[0:40000]
y_train = Y[0:40000]
x_test = X[40000:42000]
y_test = Y[40000:42000]

for i in range(X.shape[0]):
    _, f = cv2.threshold(X[i].reshape((28,28)), 80, 255, cv2.THRESH_BINARY)
    X[i] = f.reshape(784)
    X[i] = X[i]/255

Y_train = []
for i in y_train:
    Y_train.append(create_op_vec(int(i)))
Y_train = np.array(Y_train)
Y_train = Y_train.reshape(40000, 10)
Y_train = Y_train.T

Y_test = []
for i in y_test:
    Y_test.append(create_op_vec(int(i)))
Y_test = np.array(Y_test)
Y_test = Y_test.reshape(2000, 10)
Y_test = Y_test.T

In [ ]:
# cost function

In [ ]:
def cost_function(parameters, A, Y, lambd):
    m = Y.shape[1]
    F = Y*np.log(A) + (1-Y)*np.log(1-A)
    J = -np.sum(np.sum(F, axis = 1))/m
    parasum = 0
    L = int(len(parameters)/2)
    for l in range(1, L + 1):
        parasum += np.sum(np.sum(parameters['W' + str(l)]**2))
    return J + parasum*lambd/(2*m)

In [ ]:
# FORWARD PROPAGATION

In [ ]:
def initialise_parameters(layer_dims):
    L = len(layer_dims) - 1
    
    parameters = {}
    
    for l in range(1,L + 1):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters["b" + str(l)] = np.random.randn(layer_dims[l], 1) * 0.01
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters

In [ ]:
def relu(Z):
    return np.maximum(0, Z)

In [ ]:
def sigmoid(Z):
    return 1/(1 + np.exp(-Z))

In [ ]:
def linear_jump(A_prev, W, b):
    return np.dot(W, A_prev) + b

In [ ]:
def forward_prop(parameters, X, Y, i, lambd):
    
    L = int(len(parameters)/2)
    cacheZ = []
    cacheA = []
    cacheA.append(X)
    A_prev = X
    
    for l in range(1,L):
        Z = linear_jump(A_prev, parameters["W" + str(l)], parameters["b" + str(l)])
        A_prev = relu(Z)
        cacheZ.append(Z)
        cacheA.append(A_prev)
    
    Z_final = linear_jump(A_prev, parameters["W" + str(L)], parameters["b" + str(L)])
    AL = sigmoid(Z_final)
    
    cacheZ.append(Z_final)
    
    
    if i%5 == 0:
        print("Cost at iteration " + str(i) + " = " + str(cost_function(parameters, AL, Y, lambd)))
    
    return cacheA, cacheZ, AL

In [ ]:
# BACKWARD PROPAGATAION

In [ ]:
def relu_backward(Z):
    final = np.maximum(0, Z)
    for i in range(final.shape[0]):
        for j in range(final.shape[1]):
            if final[i][j] != 0:
                final[i][j] = 1
    
    return final

In [ ]:
def sigmoid_backward(Z):
    A = sigmoid(Z)
    return A*(1-A)

In [ ]:
def back_prop(parameters, X, Y, iteration, lambd):
    
    m = X.shape[1]
    L = int(len(parameters)/2)
    cacheA, cacheZ, AL = forward_prop(parameters, X, Y, iteration, lambd)
    gradients = {}
    
    dZ = AL - Y
    gradients['dW' + str(L)] = np.dot(dZ, cacheA[L-1].T)/m + lambd * parameters["W" + str(L)] / m
    gradients['db' + str(L)] = np.sum(dZ, axis=1, keepdims=True)/m
    dA = np.dot(parameters['W' + str(L)].T, dZ)
    
    for l in reversed(range(1,L)):
        dZ = dA * relu_backward(cacheZ[l-1])
        gradients['dW' + str(l)] = np.dot(dZ, cacheA[l-1].T)/m + lambd * parameters["W" + str(l)] / m
        gradients['db' + str(l)] = np.sum(dZ, axis=1, keepdims=True)/m
        dA = np.dot(parameters['W' + str(l)].T, dZ)
        
    return gradients

In [ ]:
# UPDATING PARAMETERS

def jump(parameters, gradients, alpha):
    L = int(len(parameters)/2)
    for l in range(1, L + 1):
        parameters["W" + str(l)] -= alpha*gradients["dW" + str(l)]
        parameters["b" + str(l)] -= alpha*gradients["db" + str(l)]
        
    
    return parameters

In [ ]:
# TRAINING OUR MODEL

def train(X, Y, alpha, iterations, layer_dims, parameters, lambd):
        
    for i in tqdm(range(iterations)):
        gradients = back_prop(parameters, X, Y, i, lambd)
        parameters = jump(parameters, gradients, alpha)
    
    return parameters

In [ ]:
# SPECIFYING NEURAL NETWORK

n_x = 784
n_y = 10
n_h1 = 500
n_h2 = 500
n_h3 = 500
layer_dims = np.array([n_x, n_h1, n_h2, n_y])
r_parameters = initialise_parameters(layer_dims)

In [ ]:
# GETTING OPTIMAL PARAMETERS

params = train(x_train.T, Y_train, 0.102, 1000, layer_dims, params, 0)

  0%|          | 0/1000 [00:00<?, ?it/s]

Cost at iteration 0 = 0.18760401256972692


  0%|          | 5/1000 [02:50<9:24:43, 34.05s/it]

Cost at iteration 5 = 0.1870445813893506


  1%|          | 10/1000 [05:40<9:22:01, 34.06s/it]

...

 99%|█████████▉| 990/1000 [9:24:40<05:41, 34.13s/it]

Cost at iteration 990 = 0.10924703538087349


100%|█████████▉| 995/1000 [9:27:30<02:50, 34.11s/it]

Cost at iteration 995 = 0.10896878752048342


100%|██████████| 1000/1000 [9:30:25<00:00, 34.23s/it]


In [ ]:
 params

{'W1': array([[-4.78033428e-05,  7.65396409e-04,  1.22445836e-02, ...,
          1.75577268e-02,  2.33512010e-03, -4.10735975e-03],
        [ 5.19937869e-03, -1.77309342e-02, -1.39296903e-02, ...,
         -4.96210588e-03,  6.39777035e-04, -1.59029335e-02],
        [-6.98164303e-03, -6.69264721e-03,  7.62955594e-03, ...,
         -1.17092553e-02, -2.36001078e-02,  6.37170048e-03],
        ...,
        [ 4.60102911e-03, -1.19884865e-02,  1.20841917e-03, ...,
         -1.64973824e-03, -3.20075812e-03, -1.49853592e-02],
        [-1.34262077e-02,  1.61367275e-03,  8.47106640e-03, ...,
          4.09493553e-03,  2.13859507e-02,  1.51418694e-02],
        [ 2.90331970e-03, -1.36101609e-02, -2.18674431e-02, ...,
         -1.53202390e-02, -2.69308206e-03, -1.13398838e-03]]),
 'W2': array([[ 0.06345484,  0.02470317,  0.01672148, ...,  0.00957294,
          0.02548475,  0.01547999],
        [ 0.02291387, -0.01022143, -0.00461837, ..., -0.00115049,
          0.00112585,  0.02704834],
        [-0.0

In [ ]:
_, _, A = forward_prop(params, x_test.T, Y_test, 0, 0)
_, _, A_t = forward_prop(params, x_train.T, Y_train, 0, 0)

Cost at iteration 0 = 0.22458282824601333
Cost at iteration 0 = 0.10869141585860348


In [ ]:
pred = np.argmax(A, axis=0)
pred1 = np.argmax(A_t, axis=0)

In [ ]:
np.mean(y_test == pred)

0.967

In [ ]:
np.mean(y_train == pred1)

0.988275

In [ ]:
x_train.shape

(40000, 784)

In [ ]:
for i in range(len(pred)):
    print(str(pred[i]) + " " + str(y_test[i]))

In [ ]:
y_test.tolist()

In [ ]:
# SAVING THE PARAMETERS IN A CSV FILE

os.chdir('/content/drive/My Drive/Digit Recognizer/Parameters/')
L = int(len(params)/2)
for l in tqdm(range(1, L+1)):
    with open('params W' + str(l) + '.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(np.ones(params['W' + str(l)].shape[1]) * 2)
        writer.writerows(params['W' + str(l)])
    
    with open('params b' + str(l) + '.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(np.ones(params['b' + str(l)].shape[1]) * 2)
        writer.writerows(params['b' + str(l)])

100%|██████████| 3/3 [00:01<00:00,  2.76it/s]
